In [1]:
import pandas as pd
import numpy as np

In [11]:
data1 = pd.read_csv("D:/college/SEM 6/Internship/books.csv")
print(data1)
data2 = pd.read_csv('D:/college/SEM 6/Internship/books_new.csv')
print(data2)

                            Title              Author              Genre  \
0        Fundamentals of Wavelets    Goswami, Jaideva  signal_processing   
1                      Data Smart       Foreman, John       data_science   
2        God Created the Integers    Hawking, Stephen        mathematics   
3               Superfreakonomics     Dubner, Stephen          economics   
4                     Orientalism        Said, Edward            history   
..                            ...                 ...                ...   
206      Structure and Randomness        Tao, Terence        mathematics   
207  Image Processing with MATLAB       Eddins, Steve  signal_processing   
208                   Animal Farm      Orwell, George            fiction   
209                    Idiot, The  Dostoevsky, Fyodor            fiction   
210            Christmas Carol, A    Dickens, Charles            fiction   

     Height      Publisher  
0       228          Wiley  
1       235          Wiley  


In [25]:
print(data1.value_counts().sum())

data1.isnull().value_counts()

112


Title  Author  Genre  Height  Publisher
False  False   False  False   False        112
                              True          75
       True    False  False   True          21
                              False          3
Name: count, dtype: int64

In [28]:
df1 = data1.dropna(axis = 0, how ='any')
df1

,Title,Author,Genre,Height,Publisher
0,Fundamentals of Wavelets,"Goswami, Jaideva",signal_processing,228,Wiley
1,Data Smart,"Foreman, John",data_science,235,Wiley
2,God Created the Integers,"Hawking, Stephen",mathematics,197,Penguin
3,Superfreakonomics,"Dubner, Stephen",economics,179,HarperCollins
4,Orientalism,"Said, Edward",history,197,Penguin
...,...,...,...,...,...
114,Rationality & Freedom,"Sen, Amartya",economics,213,Springer
115,Clash of Civilizations and Remaking of the Wor...,"Huntington, Samuel",history,228,Simon&Schuster
116,Uncommon Wisdom,"Capra, Fritjof",nonfiction,197,Fontana
117,One,"Bach, Richard",nonfiction,172,Dell


In [30]:
df2 = data2.dropna(axis = 0, how = "any")
df2

,Title,Author,Genre,SubGenre,Height,Publisher
0,Fundamentals of Wavelets,"Goswami, Jaideva",tech,signal_processing,228,Wiley
1,Data Smart,"Foreman, John",tech,data_science,235,Wiley
2,God Created the Integers,"Hawking, Stephen",tech,mathematics,197,Penguin
3,Superfreakonomics,"Dubner, Stephen",science,economics,179,HarperCollins
4,Orientalism,"Said, Edward",nonfiction,history,197,Penguin
...,...,...,...,...,...,...
114,Rationality & Freedom,"Sen, Amartya",science,economics,213,Springer
115,Clash of Civilizations and Remaking of the Wor...,"Huntington, Samuel",nonfiction,history,228,Simon&Schuster
116,Uncommon Wisdom,"Capra, Fritjof",nonfiction,anthology,197,Fontana
117,One,"Bach, Richard",nonfiction,autobiography,172,Dell


In [76]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

# Assuming df2 is already defined and loaded elsewhere in your code
print(df2.head())

# Function to train the model and compute the cosine similarity matrix
def train_model(df2):
    # Combine Genre and SubGenre into a single feature
    df2['combined_features'] = df2['Genre'] + ' ' + df2['SubGenre'].fillna('')
    
    # Create a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    
    # Fit and transform the combined_features
    tfidf_matrix = tfidf_vectorizer.fit_transform(df2['combined_features'])
    
    # Compute the cosine similarity matrix
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    return df2, cosine_sim

# Function to get recommendations based on genre
def recommend_books_by_genre(df2, cosine_sim, selected_genre, num_recommendations=3):
    # Filter books by the selected genre
    genre_books = df2[df2['Genre'].str.contains(selected_genre, case=False, na=False)]
    
    if genre_books.empty:
        return f"No books found for the genre: {selected_genre}"

    # Randomly select a reference book from the genre_books
    reference_book_idx = random.choice(genre_books.index)
    reference_book_title = df2.loc[reference_book_idx, 'Title']

    # Get the pairwise similarity scores of all books with the reference book
    sim_scores = list(enumerate(cosine_sim[reference_book_idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the num_recommendations most similar books
    sim_scores = sim_scores[1:num_recommendations+1]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top most similar books
    return df2.iloc[book_indices]

# Train the model
df2, cosine_sim = train_model(df2)

# User selects a genre
selected_genre = 'tech'  # Change this to the genre of your choice

# Get recommendations
recommended_books = recommend_books_by_genre(df2, cosine_sim, selected_genre, num_recommendations=3)

# Check if recommended_books is a DataFrame or a string
if isinstance(recommended_books, str):
    print(recommended_books)
else:
    print("Recommended Books:")
    print(recommended_books[['Title', 'Author', 'Genre']])


                      Title            Author       Genre           SubGenre  \
0  Fundamentals of Wavelets  Goswami, Jaideva        tech  signal_processing   
1                Data Smart     Foreman, John        tech       data_science   
2  God Created the Integers  Hawking, Stephen        tech        mathematics   
3         Superfreakonomics   Dubner, Stephen     science          economics   
4               Orientalism      Said, Edward  nonfiction            history   

   Height      Publisher       combined_features  
0     228          Wiley  tech signal_processing  
1     235          Wiley       tech data_science  
2     197        Penguin        tech mathematics  
3     179  HarperCollins       science economics  
4     197        Penguin      nonfiction history  
Recommended Books:
                   Title             Author    Genre
21       Analysis, Vol I       Tao, Terence     tech
7   Drunkard's Walk, The  Mlodinow, Leonard  science
12    Birth of a Theorem    Villani

C:\Users\asus i5\AppData\Local\Temp\ipykernel_23328\2579031436.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['combined_features'] = df2['Genre'] + ' ' + df2['SubGenre'].fillna('')


In [124]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

# Assuming df2 is already defined and loaded elsewhere in your code
print(df2.head())

# Function to train the model and compute the cosine similarity matrix
def train_model(df2):
    # Combine Genre and SubGenre into a single feature
    df2['combined_features'] = df2['Genre'] + ' ' + df2['SubGenre'].fillna('')
    
    # Create a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    
    # Fit and transform the combined_features
    tfidf_matrix = tfidf_vectorizer.fit_transform(df2['combined_features'])
    
    # Compute the cosine similarity matrix
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    return df2, cosine_sim

# Function to get recommendations based on genre and/or subgenre
def recommend_books(df2, cosine_sim, selected_genre=None, selected_subgenre=None, num_recommendations=3):
    if selected_genre and selected_subgenre:
        genre_books = df2[df2['Genre'].str.contains(selected_genre, case=False, na=False) & 
                          df2['SubGenre'].str.contains(selected_subgenre, case=False, na=False)]
    elif selected_genre:
        genre_books = df2[df2['Genre'].str.contains(selected_genre, case=False, na=False)]
    elif selected_subgenre:
        genre_books = df2[df2['SubGenre'].str.contains(selected_subgenre, case=False, na=False)]
    else:
        return "Please select at least one genre or subgenre."

    if genre_books.empty:
        if selected_genre:
            genre_books = df2[df2['Genre'].str.contains(selected_genre, case=False, na=False)]
            if genre_books.empty:
                return f"No books found for the genre: {selected_genre}"
        if selected_subgenre:
            genre_books = df2[df2['SubGenre'].str.contains(selected_subgenre, case=False, na=False)]
            if genre_books.empty:
                return f"No books found for the subgenre: {selected_subgenre}"
        if genre_books.empty:
            return f"No books found for the genre: {selected_genre} and subgenre: {selected_subgenre}"

    # Randomly select a reference book from the genre_books
    reference_book_idx = random.choice(genre_books.index)
    reference_book_title = df2.loc[reference_book_idx, 'Title']

    # Get the pairwise similarity scores of all books with the reference book
    sim_scores = list(enumerate(cosine_sim[reference_book_idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the num_recommendations most similar books
    sim_scores = sim_scores[1:num_recommendations+1]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top most similar books
    return df2.iloc[book_indices]

# Train the model
df2, cosine_sim = train_model(df2)

# User selects a genre and subgenre
selected_genre = ''  # Change this to the genre of your choice
selected_subgenre = 'mathematics'  # Change this to the subgenre of your choice, or None for any subgenre

# Get recommendations
recommended_books = recommend_books(df2, cosine_sim, selected_genre, selected_subgenre, num_recommendations=3)

# Check if recommended_books is a DataFrame or a string
if isinstance(recommended_books, str):
    print(recommended_books)
else:
    print("Recommended Books:")
    print(recommended_books[['Title', 'Author', 'Genre']])

                      Title            Author       Genre           SubGenre  \
0  Fundamentals of Wavelets  Goswami, Jaideva        tech  signal_processing   
1                Data Smart     Foreman, John        tech       data_science   
2  God Created the Integers  Hawking, Stephen        tech        mathematics   
3         Superfreakonomics   Dubner, Stephen     science          economics   
4               Orientalism      Said, Edward  nonfiction            history   

   Height      Publisher       combined_features  
0     228          Wiley  tech signal_processing  
1     235          Wiley       tech data_science  
2     197        Penguin        tech mathematics  
3     179  HarperCollins       science economics  
4     197        Penguin      nonfiction history  
Recommended Books:
                   Title             Author    Genre
21       Analysis, Vol I       Tao, Terence     tech
7   Drunkard's Walk, The  Mlodinow, Leonard  science
12    Birth of a Theorem    Villani

C:\Users\asus i5\AppData\Local\Temp\ipykernel_23328\2009758768.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['combined_features'] = df2['Genre'] + ' ' + df2['SubGenre'].fillna('')


In [46]:
print(df2["Genre"].value_counts())
print("\n")
print(df2["SubGenre"].value_counts())

Genre
fiction       34
nonfiction    33
tech          24
science       13
philosophy     8
Name: count, dtype: int64


SubGenre
history              18
classic              17
novel                17
economics            10
data_science          9
autobiography         7
signal_processing     5
mathematics           5
computer_science      5
physics               5
misc                  4
psychology            2
philosophy            2
poetry                1
education             1
science               1
trivia                1
objectivism           1
anthology             1
Name: count, dtype: int64
